In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Define the url we are going to take the info from

url = 'https://en.wikipedia.org/wiki/Critics%27_Choice_Movie_Award_for_Best_Picture'

In [3]:
# Make the request to the url

res = requests.get(url)

# Check if all went well

res.status_code

200

In [4]:
# 'html.parser' is used as the parser for the BeautifulSoup library.

soup = BeautifulSoup(res.content, 'html.parser') 

In [5]:
# Extract the name of the tables:

tables = soup.find_all("table", {"class": "wikitable"})

# Get the number of tables in the web, in this case 4 in this case

len(tables)

4

In [6]:
# Get the info of every table

soup1 = tables[0].find_all("td")
soup2 = tables[1].find_all("td")
soup3 = tables[2].find_all("td")
soup4 = tables[3].find_all("td")

# Put all the info together

the_soup = soup1 + soup2 + soup3 + soup4

In [7]:
# Transform the data extacted into text
# Iterating over the outer list

picture1 = [x.text for item in the_soup 

                # Iterating over the inner lists
                
               for x in item] 

In [8]:
# The re.sub pattern '\[(.*?)\]' is used to match the elements between the square brackets and and replace it with a ""

picture1 = [re.sub(r'\[(.*?)\]', '', item) for item in picture1]

# This re. sub pattern '\((.*?)\)' is used to match the elements between the parenthesis and replace it with a ""

picture1 = [re.sub(r'\((.*?)\)', '', item) for item in picture1]

# This takes out the empty spaces from our dataset

picture1 = [item for item in picture1 if item.strip("")]

In [9]:
# Initialize an empty list to store the new data

new_data1 = []

# Initialize a counter variable to keep track of the current index

i = 0

# Iterate through the original dataset

while i < len(picture1):

    # Check if the current item is ' and '

    if picture1[i] == ' and ':

        # If it is, merge it with the previous and next item

        new_data1[-1] = new_data1[-1] + picture1[i] + picture1[i+1]

        # Skip the next item (to avoid adding it again)

        i += 2
    elif picture1[i] == ' ':

         # If it is, merge it with the previous and next item

        new_data1[1] = new_data1[1] + picture1[i] + picture1[i+1]

         # Skip the next item (to avoid adding it again)

        i += 2   
    else:

        # If the current item is not ' and ', append it to the new list

        new_data1.append(picture1[i])

        # Increment the counter variable

        i += 1

In [10]:
# Iterate over the elements of the actors1 list

for i, item in enumerate(new_data1):

    # Check if the current element is 'Gladiator / Quills'

    if new_data1[i] == 'Sense and Sensibility                          ':

        # Replace the current element with 'Gladiator'

        new_data1[i] = 'Sense and Sensibility'

In [11]:
# Create a new dataframe with columns "year_ceremony", "film", and "name"

df2 = pd.DataFrame(columns=["year_ceremony", "film", "name"])

# Initialize empty variables to store the year, film, and name

year = ""
film = ""
name = ""

# Iterate through the new_data list, using the enumerate function to keep track of the index

for i, element in enumerate(new_data1):

    # Check if the element contains a newline character

    if "\n" in element: 

        # If it does, append the current year, film, and name to the dataframe

        df2 = df2.append({"year_ceremony": year, "film": film, "name": name}, ignore_index=True)

        # Update the film, name and year variables with the new values

        film = new_data1[i-2]
        name = new_data1[i-1]
        year = new_data1[i-3]
    if i == len(new_data1) - 1:

        # if it's the last element append the last year, film, and name to the dataframe

        df2 = df2.append({"year_ceremony": year, "film": film, "name": name}, ignore_index=True)

C:\Users\juano\AppData\Local\Temp\ipykernel_9348\1512279316.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_9348\1512279316.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_9348\1512279316.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_9348\1512279316.py:21: FutureWarning: The frame.append method is deprecated and wil

In [12]:
# Iterate through the elements in 'df2'

for years in range(1, len(df2)):

    # Check if the current element is a newline character

    if df2.iloc[years]['year_ceremony'] == '\n':

        # Replace the current element with the value of the previous element

        df2.at[years, 'year_ceremony'] = df2.iloc[years-1]['year_ceremony']

In [13]:
# Drop the first row (index 0) from the DataFrame because is empty

df2 = df2.drop(df2.index[0])

In [14]:
# Use the assign method to add a new column called "category" and the award name "cca"

df2 = df2.assign(category='Best Picture').assign(cca='Nominee')

# Create a list of columns in the desired order

columns = ['year_ceremony', 'category', 'film', 'name', 'cca']

# Reorder columns

df2 = df2[columns]

In [15]:
# Extract the info of the winners:

cca_winners = soup.find_all("td", {"style":"background:#B0C4DE;"})

# Get the number of the award winners

len(cca_winners)

56

In [16]:
# Extract the text from cca_winners

cca_winners = [x.text for x in cca_winners]

# Split by "\n" and get the element first element

cca_winners= [x.split("\n")[0] for x in cca_winners]

In [17]:
# Iterate for every element in the cca_winners

for match in cca_winners:
    
    # If a match is found between the list and the columns, change from Nominnee to Winner
    
    df2.loc[df2["film"] == match, 'cca'] = 'Winner' 

In [18]:
# Convert to csv and store it in data folder

df2.to_csv("../data/Best Picture_soup_cca.csv")